In [3]:
import pandas as pd
import numpy as np
df_tournage = pd.read_csv("src/lieux-de-tournage-a-paris.csv", delimiter=';', on_bad_lines='skip')

# Drop unused :
df_tournage = df_tournage.drop(columns=["Identifiant du lieu","Localisation de la scène","Année du tournage","Coordonnée en X", "Coordonnée en Y","geo_shape"])

# Corrections Type :
df_tournage["Code postal"] = pd.to_numeric(df_tournage["Code postal"])
df_tournage = df_tournage.dropna(subset = ['Code postal'])
df_tournage["Code postal"] = df_tournage["Code postal"].astype(int)
df_tournage["Date de début"] = pd.to_datetime(df_tournage["Date de début"])
df_tournage["Date de fin"] = pd.to_datetime(df_tournage["Date de fin"])
df_tournage["latitude"] = pd.to_numeric( [ x.split(",")[0] for x in df_tournage["geo_point_2d"][:][:] ] )
df_tournage["longitude"] = pd.to_numeric( [ x.split(",")[1] for x in df_tournage["geo_point_2d"][:][:] ] )
df_tournage = df_tournage.drop(columns="geo_point_2d")

# Clear Name Columns :
df_tournage = df_tournage.rename(columns = {"Type de tournage":"type", "Code postal":"postal", "Date de début":"debut", "Date de fin":"fin"})
df_tournage.columns = [ x.lower() for x in df_tournage.columns.tolist() ] # ABC -> abc

# keep only films :
df_tournage = df_tournage[df_tournage["type"] == 'Long métrage']
df_tournage = df_tournage.drop(columns= ['type'])

df_tournage = df_tournage.reset_index().drop(columns=['index'])

df_tournage.head()

,titre,réalisateur,producteur,postal,debut,fin,latitude,longitude
0,TOUT S'EST BIEN PASSE,Francois OZON,MANDARIN PRODUCTION,75013,2020-08-20,2020-08-21,48.835660,2.348315
1,Une jeune fille qui va bien,Sandrine Kiberlain,CURIOSA FILMS,75004,2020-08-31,2020-09-01,48.854533,2.361694
2,French Exit,Azazel Jacobs,Same Player,75012,2019-12-04,2019-12-04,48.850067,2.376519
3,FIN DE MATINEE,Hiroshi NISHATANI,COMME DES CINEMAS,75004,2018-11-05,2018-11-05,48.854112,2.354679
4,HORS NORMES,Eric Toledano et Olivier Nakache,ADNP QUAD FILMS,75001,2018-11-05,2018-11-06,48.865744,2.327446


In [77]:

df_imdb = pd.read_json("src/imdb_films_infos.json")

In [4]:
import re

pd.options.display.max_rows = None
pd.options.display.max_seq_items = None

df_imdb['date'] = df_imdb['date'].str.replace(r'\D+', '')



for i in range(len(df_imdb)): # 
    raw = df_imdb.iloc[i]

    values = [ raw['duree'], raw['genre'], raw['parental_advisor'] ]
    
    df_imdb.genre[i] = None
    df_imdb.duree[i] = None
    df_imdb.parental_advisor[i] = None

    for item in values:

        if type(item) is str :
            if '\n' in item:
                df_imdb.genre[i] = item
            elif 'min' in item:
                df_imdb.duree[i] = item
            else :
                df_imdb.parental_advisor[i] = item


NameError: name 'df_imdb' is not defined

In [79]:
df_imdb['date'] = [ x[-4:] for x in df_imdb['date']] # Dernière date 

df_imdb['duree'] = df_imdb['duree'].str.replace(r'\D+', '')

df_imdb['genre'] = df_imdb['genre'].str.replace(r'\n', '')

df_imdb['resume'] = df_imdb['resume'].str.replace(r'\n', '')

df_imdb.head(5)

<ipython-input-79-c771b306740b>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_imdb['duree'] = df_imdb['duree'].str.replace(r'\D+', '')
<ipython-input-79-c771b306740b>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_imdb['genre'] = df_imdb['genre'].str.replace(r'\n', '')
<ipython-input-79-c771b306740b>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_imdb['resume'] = df_imdb['resume'].str.replace(r'\n', '')


,imdb_search,title,date,parental_advisor,duree,genre,imdb_note,metascore,vote,resume
0,https://www.imdb.com/search/title/?title=TOUT+...,Everything Went Fine,2021,Tous publics,113,Drama,6.8,67.0,"1,572","When André, 85, has a stroke, Emmanuelle hurri..."
1,https://www.imdb.com/search/title/?title=Une+j...,A Radiant Girl,2021,Tous publics,98,"Drama, History",6.1,NaN,165,"Paris, summer 1942. Irene is Jewish and French..."
2,https://www.imdb.com/search/title/?title=Frenc...,French Exit,2020,R,113,"Comedy, Drama",5.9,56.0,"7,154",An aging Manhattan socialite existing on the l...
3,https://www.imdb.com/search/title/?title=HORS+...,The Specials,2019,Tous publics,114,"Comedy, Drama",7.4,NaN,"9,086","The story of two men, educators of children an..."
4,https://www.imdb.com/search/title/?title=DOCTE...,A Good Doctor,2019,Tous publics,90,Comedy,6.5,NaN,"1,827","On Christmas Eve, Serge is the only on-call em..."


In [5]:
df_imdb.dtypes

df_imdb['vote'] = [ '0' if x == None else x for x in df_imdb['vote']]
df_imdb['vote'] = df_imdb['vote'].str.replace(r'\D+', '').str.replace(',', '')
df_imdb['vote'] = df_imdb['vote'].astype(int)
df_imdb['vote'] = [ np.nan if x == 0 else x for x in df_imdb['vote']]

df_imdb['date'] = [ '0' if x == None else x for x in df_imdb['date']]
df_imdb['date'] = df_imdb['date'].astype(int)
df_imdb['date'] = [ np.nan if x == 0 else x for x in df_imdb['date']]

df_imdb['duree'] = [ '0' if x == None else x for x in df_imdb['duree']]
df_imdb['duree'] = df_imdb['duree'].astype(int)
df_imdb['duree'] = [ np.nan if x == 0 else x for x in df_imdb['duree']]

df_imdb = df_imdb.drop_duplicates(subset='imdb_search', keep="first")
df_imdb = df_imdb.reset_index().drop(columns=["index"])

NameError: name 'df_imdb' is not defined

In [1]:
df_imdb.head()

NameError: name 'df_imdb' is not defined

In [84]:
df_imdb.to_csv(r'export/df_imdb_scrap.csv', index = False, header=True)